# 05 · Submit — упаковка финального сабмита 🚀

Берём `ranked.parquet` из 04_blend_eval → проверяем покрытие/формат → стабильная сортировка и ранги → конвертация
в формат соревнования → подпись/версирование → zip (если нужно) → (опц.) автосабмит.

Артефакты сабмита: `artifacts/recsys/submits/<dataset>/<profile>/<SUBMIT_ID>/...`


In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [ ]:
import os, sys, json, time, math, warnings, hashlib, zipfile, random
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
from IPython.display import display, clear_output, HTML
import ipywidgets as W

warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth", 140)

REPO = Path.cwd()

def mem_gb(obj=None):
    try:
        import psutil
        if obj is None: return psutil.Process().memory_info().rss/(1024**3)
    except Exception:
        pass
    try:
        if hasattr(obj, "memory_usage"):
            return float(obj.memory_usage(deep=True).sum())/(1024**3)
        return float(np.array(obj).nbytes)/(1024**3)
    except Exception:
        return float("nan")

def save_json(path: Path, obj: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2))

def now_tag(): 
    return datetime.now().strftime("%m%d_%H%M")

def short_hash(s: str, n=8) -> str:
    return hashlib.sha1(s.encode()).hexdigest()[:n]

def art_paths(dataset_id: str, profile_path: str):
    prof = Path(profile_path).stem
    root = Path("artifacts")
    return dict(
        dataio  = root/"recsys"/"dataio"/dataset_id/prof,
        rankers = root/"recsys"/"ranker"/dataset_id/prof,
        ensemble= root/"recsys"/"ensemble"/dataset_id/prof,
        submits = root/"recsys"/"submits"/dataset_id/prof,
        profile = prof
    )

def read_parquet_safe(path: Path, cols: List[str] = None):
    df = pd.read_parquet(path)
    if cols:
        miss = [c for c in cols if c not in df.columns]
        if miss:
            raise AssertionError(f"{path} не содержит столбцы: {miss}")
        return df[cols].copy()
    return df

def fingerprint(path: Path):
    h1 = hashlib.sha1(); h2 = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h1.update(chunk); h2.update(chunk)
    return dict(sha1=h1.hexdigest(), md5=h2.hexdigest(), bytes=path.stat().st_size)


In [ ]:
BTN_LAYOUT = W.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_LAYOUT = W.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_LAYOUT = W.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")

In [ ]:
STATE = dict(
    DATASET_ID   = "s5e11",
    PROFILE_PATH = "configs/recsys/profiles/gate.yaml",
    K_OUT        = 100,
    QUERY_COL    = "query_id",
    ITEM_COL     = "item_id",
    RANK_COL     = "rank",
    SUBMIT_FORMAT= "pairs_csv",     # pairs_csv | space_separated | jsonl | custom
    STRICT_SCHEMA= True,
    STABLE_TIEBREAK = True,
    COMPRESS_ZIP = True,
    RANDOM_SEED  = 42,
    CUSTOM_SCHEMA_PATH = "configs/recsys/submission_schema.json",  # для custom
    KAGGLE_COMP  = "",              # опционально: id соревнования для Kaggle API
)

w_ds   = W.Text(STATE["DATASET_ID"], description="dataset_id:", layout=W.Layout(width="240px"))
w_prof = W.Text(STATE["PROFILE_PATH"], description="profile:", layout=W.Layout(width="520px"))
w_kout = W.IntText(STATE["K_OUT"], description="K_OUT:", layout=W.Layout(width="160px"))
w_qc   = W.Text(STATE["QUERY_COL"], description="QUERY_COL:", layout=W.Layout(width="220px"))
w_ic   = W.Text(STATE["ITEM_COL"], description="ITEM_COL:", layout=W.Layout(width="220px"))
w_rc   = W.Text(STATE["RANK_COL"], description="RANK_COL:", layout=W.Layout(width="220px"))
w_fmt  = W.Dropdown(options=["pairs_csv","space_separated","jsonl","custom"], value=STATE["SUBMIT_FORMAT"], description="format:")
w_str  = W.Checkbox(STATE["STRICT_SCHEMA"], description="strict_schema")
w_tie  = W.Checkbox(STATE["STABLE_TIEBREAK"], description="stable_tiebreak")
w_zip  = W.Checkbox(STATE["COMPRESS_ZIP"], description="compress_zip")
w_seed = W.IntText(STATE["RANDOM_SEED"], description="seed:")
w_schema = W.Text(STATE["CUSTOM_SCHEMA_PATH"], description="custom_schema:", layout=W.Layout(width="520px"))
w_kaggle = W.Text(STATE["KAGGLE_COMP"], description="kaggle_comp:", layout=W.Layout(width="320px"))

btn_scan = W.Button(description="Сканировать ансамбли", button_style="primary", layout=BTN_LAYOUT)
out_scan = W.Output()
SEL = dict(ens=[], widget=None)

def list_ensemble_runs(dataset_id, profile_path) -> List[Path]:
    A = art_paths(dataset_id, profile_path)
    d = A["ensemble"]
    if not d.exists(): return []
    runs = [p for p in sorted(d.iterdir()) if p.is_dir()]
    # только те, где есть ranked.parquet
    return [r for r in runs if (r/"ranked.parquet").exists()]

def on_scan(_):
    with out_scan:
        clear_output()
        # обновим состояние
        STATE.update(
            DATASET_ID=w_ds.value.strip(), PROFILE_PATH=w_prof.value.strip(),
            K_OUT=int(w_kout.value), QUERY_COL=w_qc.value.strip(), ITEM_COL=w_ic.value.strip(),
            RANK_COL=w_rc.value.strip(), SUBMIT_FORMAT=w_fmt.value, STRICT_SCHEMA=bool(w_str.value),
            STABLE_TIEBREAK=bool(w_tie.value), COMPRESS_ZIP=bool(w_zip.value), RANDOM_SEED=int(w_seed.value),
            CUSTOM_SCHEMA_PATH=w_schema.value.strip(), KAGGLE_COMP=w_kaggle.value.strip()
        )
        ens = list_ensemble_runs(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        if not ens:
            print("Не найдено ансамблей. Сначала собери в 04_blend_eval.")
            return
        options = [(f"{e.name}", str(e)) for e in ens]
        SEL["ens"]=ens
        SEL["widget"]= W.Select(options=options, description="ens_id:", rows=min(len(options), 12), layout=W.Layout(width="520px"))
        display(SEL["widget"])

W.VBox([
    W.HTML("<h3>Параметры сабмита</h3>"),
    W.HBox([w_ds, w_seed, w_kout], layout=ROW_LAYOUT),
    w_prof,
    W.HBox([w_qc, w_ic, w_rc], layout=ROW_LAYOUT),
    W.HBox([w_fmt, w_str, w_tie, w_zip], layout=ROW_LAYOUT),
    w_schema,
    w_kaggle,
    btn_scan, out_scan
])

btn_scan.on_click(on_scan)


In [ ]:
btn_load = W.Button(description="Загрузить выбранный ансамбль", button_style="success", layout=BTN_LAYOUT)
out_load = W.Output()

DATA = dict(ranked=None, scores=None, ens_dir=None, test_queries=None, blender_config=None)

def try_load_test_queries(A) -> Optional[pd.Series]:
    # Пытаемся найти список тест-queries из dataio-папки
    candidates = [
        A["dataio"]/ "test_queries.parquet",
        A["dataio"]/ "test_queries.csv",
        A["dataio"]/ "test.parquet",
        A["dataio"]/ "test.csv",
    ]
    for p in candidates:
        if p.exists():
            df = pd.read_parquet(p) if p.suffix==".parquet" else pd.read_csv(p)
            # ищем колонку query_id
            qcol = STATE["QUERY_COL"]
            if qcol not in df.columns:
                # fallback: попытка найти что-то похожее
                for c in df.columns:
                    if "query" in c.lower():
                        qcol = c; break
            if qcol not in df.columns:
                continue
            return df[qcol].astype(str)
    return None

def on_load(_):
    with out_load:
        clear_output()
        if not SEL.get("widget"):
            print("Сначала отсканируй ансамбли и выбери один.")
            return
        ens_dir = Path(SEL["widget"].value)
        ranked_path = ens_dir/"ranked.parquet"
        scores_path = ens_dir/"scores_blend.parquet"
        config_path = ens_dir/"blender_config.json"
        if not ranked_path.exists():
            print(f"Нет ranked.parquet в {ens_dir}")
            return
        ranked = pd.read_parquet(ranked_path)
        ranked[STATE["QUERY_COL"]] = ranked[STATE["QUERY_COL"]].astype(str)
        ranked[STATE["ITEM_COL"]]  = ranked[STATE["ITEM_COL"]].astype(str)
        print("ranked:", ranked.shape, "mem:", round(mem_gb(ranked), 3), "GB")
        display(ranked.head(3))
        scores = None
        if scores_path.exists():
            scores = pd.read_parquet(scores_path)
            print("scores_blend:", scores.shape, "mem:", round(mem_gb(scores), 3), "GB")
        blender_config = json.loads(config_path.read_text()) if config_path.exists() else {}
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        test_q = try_load_test_queries(A)

        DATA.update(ranked=ranked, scores=scores, ens_dir=str(ens_dir), test_queries=test_q, blender_config=blender_config)
        if test_q is not None:
            print("test_queries найдено:", len(test_q.unique()))
        else:
            print("test_queries не найдены — проверим покрытие по имеющемуся ranked.")

btn_load.on_click(on_load)
W.VBox([btn_load, out_load])


In [ ]:
btn_fix = W.Button(description="Проверить/исправить покрытие и ранги", button_style="warning", layout=BTN_LAYOUT)
out_fix = W.Output()

FIXED = dict(df=None, report=None)

def build_fallback_pop(ranked_df: pd.DataFrame) -> List[str]:
    # Глобальная популярность item_id по встречаемости в ранках
    pop = ranked_df.groupby(STATE["ITEM_COL"]).size().sort_values(ascending=False).index.astype(str).tolist()
    return pop

def ensure_k_per_query(df: pd.DataFrame, all_queries: Optional[pd.Series], K: int) -> Tuple[pd.DataFrame, dict]:
    qcol, icol = STATE["QUERY_COL"], STATE["ITEM_COL"]
    # дедуп по (q,i)
    df = df.drop_duplicates(subset=[qcol, icol]).copy()
    # если нет полного списка queries — берём из df
    if all_queries is None:
        all_queries = df[qcol].astype(str).drop_duplicates().sort_values()
    else:
        all_queries = all_queries.astype(str).drop_duplicates().sort_values()

    # fallback pool
    fallback_items = build_fallback_pop(df)
    pool = np.array(fallback_items, dtype=str)

    rows_out = []
    added = 0
    missing_q = 0
    for q in all_queries:
        part = df.loc[df[qcol]==q, [qcol, icol, "score"]]
        if part.empty:
            # целиком отсутствует query → заполним top-K популярными
            take = pool[:K]
            rows_out.append(pd.DataFrame({qcol:q, icol:take, "score":np.linspace(0.0, -1e-6*(K-1), K)}))
            added += K; missing_q += 1
            continue
        # если меньше K — дополним
        have = set(part[icol].astype(str).tolist())
        need = K - len(part)
        if need>0:
            fill = [it for it in pool if it not in have][:need]
            if fill:
                filler = pd.DataFrame({qcol:q, icol:fill, "score":np.linspace(part["score"].min()-1e-6, part["score"].min()-1e-6*need, need)})
                part = pd.concat([part, filler], axis=0, ignore_index=True)
                added += need
        # если больше K — обрежем позже на стадии сортировки
        rows_out.append(part)
    fixed = pd.concat(rows_out, axis=0, ignore_index=True)
    return fixed, dict(added=int(added), missing_queries=int(missing_q), total_queries=int(len(all_queries)))

def stable_sort_and_rank(df: pd.DataFrame, K: int) -> pd.DataFrame:
    qcol, icol = STATE["QUERY_COL"], STATE["ITEM_COL"]
    # стабильный тай-брейк: (-score, item_id)
    df = df.copy()
    df[icol] = df[icol].astype(str)
    df = df.sort_values([qcol, "score", icol], ascending=[True, False, True])
    df[STATE["RANK_COL"]] = df.groupby(qcol)["score"].rank(ascending=False, method="first").astype("int32")
    df = df[df[STATE["RANK_COL"]]<=K]
    return df[[qcol, icol, "score", STATE["RANK_COL"]]].reset_index(drop=True)

def on_fix(_):
    with out_fix:
        clear_output()
        if DATA.get("ranked") is None:
            print("Сначала загрузите ансамбль.")
            return
        df = DATA["ranked"].copy()
        qcol, icol = STATE["QUERY_COL"], STATE["ITEM_COL"]
        # базовые проверки типов
        df[qcol] = df[qcol].astype(str); df[icol] = df[icol].astype(str)
        # покрытие и доукомплектование
        fixed, rep = ensure_k_per_query(df, DATA["test_queries"], STATE["K_OUT"])
        # сортировка/ранги
        fixed = stable_sort_and_rank(fixed, STATE["K_OUT"])
        # sanity
        byq = fixed.groupby(qcol).size()
        ok = int((byq==STATE["K_OUT"]).mean()*100)
        dup_in_query = (fixed.duplicated(subset=[qcol, icol]).sum())
        FIXED.update(df=fixed, report=dict(coverage_ok=ok, added=rep["added"], missing_queries=rep["missing_queries"], total_queries=rep["total_queries"], dup_pairs=int(dup_in_query)))
        print("coverage_ok%:", ok, "| added:", rep["added"], "| missing_queries:", rep["missing_queries"], "| total_queries:", rep["total_queries"], "| dup_pairs:", dup_in_query)
        display(fixed.head(5))

btn_fix.on_click(on_fix)
W.VBox([btn_fix, out_fix])


In [ ]:
btn_fmt = W.Button(description="Сформировать файл сабмита в выбранном формате", button_style="info", layout=BTN_LAYOUT)
out_fmt = W.Output()

SUBMIT = dict(df=None, path=None, format=None)

def to_pairs_csv(df: pd.DataFrame) -> pd.DataFrame:
    return df[[STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]]].copy()

def to_space_separated(df: pd.DataFrame) -> pd.DataFrame:
    qcol, icol, rcol = STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]
    # гарантируем порядок: по рангу
    d = df.sort_values([qcol, rcol])
    agg = d.groupby(qcol)[icol].apply(lambda s: " ".join(map(str, s.tolist()))).reset_index()
    agg.columns = [qcol, "items"]
    return agg

def write_jsonl(df: pd.DataFrame, path: Path):
    qcol, icol, rcol = STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]
    d = df.sort_values([qcol, rcol])
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        cur_q = None; bucket=[]
        for row in d.itertuples(index=False):
            q = getattr(row, qcol); i = getattr(row, icol); 
            if cur_q is None: cur_q = q
            if q != cur_q:
                f.write(json.dumps({"query_id": str(cur_q), "items": bucket}, ensure_ascii=False) + "
")
                cur_q = q; bucket=[]
            bucket.append(str(i))
        if cur_q is not None:
            f.write(json.dumps({"query_id": str(cur_q), "items": bucket}, ensure_ascii=False) + "
")

def apply_custom_schema(df: pd.DataFrame, schema_path: Path) -> Tuple[pd.DataFrame, Optional[str]]:
    # Ожидаем JSON вида: {"type":"pairs_csv"|"space_separated"|"jsonl","fields":{"query":"query_id","item":"item_id","rank":"rank","pred_str":"PredictionString"}}
    spec = json.loads(schema_path.read_text())
    t = spec.get("type","pairs_csv")
    fields = spec.get("fields",{})
    if t=="pairs_csv":
        d = df[[STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]]].copy()
        d.columns = [fields.get("query","query_id"), fields.get("item","item_id"), fields.get("rank","rank")]
        return d, "csv"
    if t=="space_separated":
        agg = to_space_separated(df)
        agg.columns = [fields.get("query","query_id"), fields.get("pred_str","PredictionString")]
        return agg, "csv"
    if t=="jsonl":
        # будем сохранять отдельно в write_jsonl
        return df.copy(), "jsonl"
    return df.copy(), "csv"

def on_fmt(_):
    with out_fmt:
        clear_output()
        if FIXED.get("df") is None:
            print("Сначала проверьте/исправьте покрытие.")
            return
        df = FIXED["df"].copy()
        SUBMIT["format"] = STATE["SUBMIT_FORMAT"]
        SUBMIT["df"] = None; SUBMIT["path"]=None
        if STATE["SUBMIT_FORMAT"]=="pairs_csv":
            SUBMIT["df"]=to_pairs_csv(df)
            display(SUBMIT["df"].head(3))
        elif STATE["SUBMIT_FORMAT"]=="space_separated":
            SUBMIT["df"]=to_space_separated(df)
            display(SUBMIT["df"].head(3))
        elif STATE["SUBMIT_FORMAT"]=="jsonl":
            # покажем предпросмотр первых 2х строк
            tmp = to_space_separated(df).head(2)
            display(tmp)
        elif STATE["SUBMIT_FORMAT"]=="custom":
            schema_path = Path(STATE["CUSTOM_SCHEMA_PATH"])
            if not schema_path.exists():
                print("custom schema не найден:", schema_path); return
            out_df, kind = apply_custom_schema(df, schema_path)
            SUBMIT["df"]=out_df; SUBMIT["format"]=("jsonl" if kind=="jsonl" else "pairs_csv")
            display(out_df.head(3))
        else:
            print("Неизвестный формат:", STATE["SUBMIT_FORMAT"])

btn_fmt.on_click(on_fmt)
W.VBox([btn_fmt, out_fmt])


In [ ]:
btn_check = W.Button(description="Жёсткие санити-чеки", button_style="danger", layout=BTN_LAYOUT)
out_check = W.Output()

def hard_checks(df: pd.DataFrame):
    qcol, icol, rcol = STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]
    issues = []
    # дубликаты внутри запроса
    dups = df.duplicated(subset=[qcol, icol]).sum()
    if dups>0: issues.append(f"Дубликаты (q,i): {dups}")
    # корректная монотонность рангов
    if rcol in df.columns:
        wrong = df.groupby(qcol)[rcol].apply(lambda s: (s.sort_values().values != np.arange(1, len(s)+1)).any()).sum()
        if wrong>0: issues.append(f"Некорректная монотонность/пробелы в рангах у {wrong} запросов")
    # NaN
    nan_any = df[[qcol]].isna().sum()[0] + (df[[icol]].isna().sum()[0] if icol in df.columns else 0)
    if nan_any>0: issues.append(f"NaN в ID: {nan_any}")
    # типы
    if df[qcol].dtype!=object:
        issues.append("QUERY_COL не string — лучше привести к str")
    return issues

def on_check(_):
    with out_check:
        clear_output()
        if SUBMIT.get("df") is None and STATE["SUBMIT_FORMAT"]!="jsonl":
            print("Сначала сформируйте файл сабмита.")
            return
        df = SUBMIT["df"] if STATE["SUBMIT_FORMAT"]!="jsonl" else FIXED["df"]
        issues = hard_checks(df)
        if issues and STATE["STRICT_SCHEMA"]:
            print("Ошибки схемы:")
            for s in issues: print(" •", s)
            raise AssertionError("STRICT_SCHEMA: true — исправьте ошибки и повторите.")
        else:
            print("Sanity OK (или предупреждения ниже при STRICT=false):")
            for s in issues: print("warn:", s)

btn_check.on_click(on_check)
W.VBox([btn_check, out_check])


In [ ]:
btn_cmp = W.Button(description="Сравнить с прошлыми сабмитами", button_style="", layout=BTN_LAYOUT)
out_cmp = W.Output()

def load_last_submits(A, max_n=5) -> List[Path]:
    d = A["submits"]
    if not d.exists(): return []
    runs = sorted([p for p in d.iterdir() if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
    return runs[:max_n]

def parse_submit_generic(path: Path, format_hint: Optional[str]=None) -> pd.DataFrame:
    # Возвращаем стандартизированный DF: query_id, item_id, rank
    qcol, icol, rcol = STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]
    # ищем файл по паттерну
    cand = None
    for fn in ["submission.csv","submission.txt","submission.jsonl"]:
        if (path/fn).exists(): cand = path/fn; break
    if cand is None:
        # попробуем любой csv
        cands = list(path.glob("*.csv")) + list(path.glob("*.txt")) + list(path.glob("*.jsonl"))
        if not cands: return pd.DataFrame(columns=[qcol, icol, rcol])
        cand = cands[0]
    if cand.suffix in [".csv",".txt"]:
        df = pd.read_csv(cand)
        cols = [c.lower() for c in df.columns]
        if rcol in df.columns or "rank" in cols:
            # пары
            # попытаемся привести имена
            df = df.rename(columns={list(df.columns)[cols.index("rank")]: rcol} if "rank" in cols else {})
            # map query/item
            if qcol not in df.columns:
                for c in df.columns:
                    if "query" in c.lower(): df = df.rename(columns={c: qcol}); break
            if icol not in df.columns:
                for c in df.columns:
                    if "item" in c.lower() or "product" in c.lower(): df = df.rename(columns={c: icol}); break
            return df[[qcol, icol, rcol]].astype({qcol:str, icol:str})
        else:
            # вероятно space-separated
            qc = qcol if qcol in df.columns else list(df.columns)[0]
            pc = "PredictionString" if "predictionstring" in cols else list(df.columns)[1]
            out_rows=[]
            for r in df.itertuples(index=False):
                q = str(getattr(r, qc)); items = str(getattr(r, pc)).split()
                for k,it in enumerate(items, start=1):
                    out_rows.append((q, str(it), k))
            return pd.DataFrame(out_rows, columns=[qcol, icol, rcol])
    elif cand.suffix==".jsonl":
        rows=[]
        with open(cand, "r", encoding="utf-8") as f:
            for line in f:
                obj = json.loads(line)
                q = str(obj.get("query_id") or obj.get("query") or obj.get(qcol))
                items = obj.get("items") or obj.get("predictions") or []
                for k,it in enumerate(items, start=1):
                    rows.append((q, str(it), k))
        return pd.DataFrame(rows, columns=[qcol, icol, rcol])
    return pd.DataFrame(columns=[qcol, icol, rcol])

def jaccard_at_k(df_a, df_b, K: int) -> float:
    qcol, icol, rcol = STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]
    a = df_a[df_a[rcol]<=K].groupby(qcol)[icol].apply(lambda s: set(s.values))
    b = df_b[df_b[rcol]<=K].groupby(qcol)[icol].apply(lambda s: set(s.values))
    qs = sorted(set(a.index) & set(b.index))
    if not qs: return float("nan")
    vals=[]
    for q in qs:
        u = a[q] | b[q]; i = a[q] & b[q]
        vals.append(len(i)/max(1,len(u)))
    return float(np.mean(vals))

def on_cmp(_):
    with out_cmp:
        clear_output()
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        prev = load_last_submits(A, max_n=5)
        if not prev:
            print("Прошлых сабмитов не найдено.")
            return
        # текущий в стандартизованном виде
        cur = FIXED["df"].copy()
        qcol, icol, rcol = STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]
        cur = cur[[qcol, icol, rcol]].astype({qcol:str, icol:str})
        rows=[]
        for p in prev:
            dfp = parse_submit_generic(p)
            if len(dfp)==0: 
                rows.append(dict(submit=p.name, jaccard_atK=np.nan)); continue
            jac = jaccard_at_k(cur, dfp, STATE["K_OUT"])
            rows.append(dict(submit=p.name, jaccard_atK=jac))
        res = pd.DataFrame(rows).sort_values("submit", ascending=False)
        display(res)

btn_cmp.on_click(on_cmp)
W.VBox([btn_cmp, out_cmp])


In [ ]:
btn_save = W.Button(description="Сохранить/подписать сабмит (+zip)", button_style="success", layout=BTN_LAYOUT)
out_save = W.Output()

PATHS = dict(submit_dir=None, submit_file=None, zip_file=None, manifest=None, submit_id=None)

def on_save(_):
    with out_save:
        clear_output()
        if FIXED.get("df") is None:
            print("Нет FIXED.df — сначала шаги выше.")
            return
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        A["submits"].mkdir(parents=True, exist_ok=True)
        ens_dir = Path(DATA["ens_dir"])
        ens_id = ens_dir.name

        # SUBMIT_ID
        fmt = STATE["SUBMIT_FORMAT"]
        tag = now_tag()
        h = short_hash(f"{ens_id}_{fmt}_{STATE['K_OUT']}_{tag}")
        submit_id = f"{tag}_{ens_id}_{fmt}_{STATE['K_OUT']}_{h}"
        out_dir = A["submits"]/submit_id
        out_dir.mkdir(parents=True, exist_ok=True)

        # Сохранение файла сабмита
        if STATE["SUBMIT_FORMAT"]=="pairs_csv":
            df = SUBMIT["df"].copy()
            file_path = out_dir/"submission.csv"
            df.to_csv(file_path, index=False)
        elif STATE["SUBMIT_FORMAT"]=="space_separated":
            df = SUBMIT["df"].copy()
            file_path = out_dir/"submission.csv"
            df.to_csv(file_path, index=False)
        elif STATE["SUBMIT_FORMAT"]=="jsonl":
            file_path = out_dir/"submission.jsonl"
            write_jsonl(FIXED["df"], file_path)
        elif STATE["SUBMIT_FORMAT"]=="custom":
            # custom → могли получить df или jsonl
            if SUBMIT["format"]=="jsonl":
                file_path = out_dir/"submission.jsonl"
                write_jsonl(FIXED["df"], file_path)
            else:
                df = SUBMIT["df"].copy()
                file_path = out_dir/"submission.csv"
                df.to_csv(file_path, index=False)
        else:
            raise ValueError("Неизвестный формат сабмита")

        # manifest
        manifest = dict(
            submit_id = submit_id,
            dataset_id = STATE["DATASET_ID"],
            profile = STATE["PROFILE_PATH"],
            ensemble_id = ens_id,
            format = STATE["SUBMIT_FORMAT"],
            K_OUT = STATE["K_OUT"],
            file = str(file_path),
            file_fp = fingerprint(file_path),
            blender_config = DATA["blender_config"],
            created_utc = datetime.utcnow().isoformat()+"Z"
        )
        save_json(out_dir/"manifest.json", manifest)

        # readme
        (out_dir/"readme.txt").write_text(
 f"""SUBMIT_ID: {submit_id}
Ensemble:  {ens_id}
Format:    {STATE['SUBMIT_FORMAT']}
K_OUT:     {STATE['K_OUT']}

How to reproduce:
  1) Run 04_blend_eval to produce {ens_id}
  2) Run 05_submit with the same dataset/profile/K_OUT and ENS={ens_id}
"""
        )

        zpath = None
        if STATE["COMPRESS_ZIP"]:
            zpath = out_dir/"submission.zip"
            with zipfile.ZipFile(zpath, "w", compression=zipfile.ZIP_DEFLATED) as zf:
                zf.write(file_path, arcname=file_path.name)
                zf.write(out_dir/"manifest.json", arcname="manifest.json")

        PATHS.update(submit_dir=str(out_dir), submit_file=str(file_path), zip_file=(None if zpath is None else str(zpath)), manifest=str(out_dir/"manifest.json"), submit_id=submit_id)
        print("✓ saved:", file_path)
        if zpath: print("✓ zipped:", zpath)

btn_save.on_click(on_save)
W.VBox([btn_save, out_save])


In [ ]:
btn_kaggle = W.Button(description="(Опц.) Kaggle submit через CLI", button_style="", layout=BTN_LAYOUT)
out_kaggle = W.Output()

def on_kaggle(_):
    with out_kaggle:
        clear_output()
        if not PATHS.get("submit_file"):
            print("Нет файла сабмита — сохраните сначала.")
            return
        if not STATE["KAGGLE_COMP"]:
            print("KAGGLE_COMP пуст. Укажите id соревнования (например, 'titanic').")
            return
        file_to_send = Path(PATHS["submit_file"])
        comp = STATE["KAGGLE_COMP"]
        msg = PATHS["submit_id"]
        print("Команда (выполните вручную, если CLI недоступен в окружении):")
        print(f"!kaggle competitions submit -c {comp} -f {file_to_send} -m "{msg}"")

btn_kaggle.on_click(on_kaggle)
W.VBox([btn_kaggle, out_kaggle])


In [ ]:
btn_panic = W.Button(description="⚡ PANIC: быстро упаковать pairs_csv из ranked", button_style="danger", layout=BTN_LAYOUT)
out_panic = W.Output()

def on_panic(_):
    with out_panic:
        clear_output()
        if DATA.get("ranked") is None:
            print("Нет ranked — загрузите ансамбль.")
            return
        # Пересоберём фиксированный набор напрямую из ranked (без доукомплектования, только сорт/ранги)
        df = DATA["ranked"].copy()
        df[STATE["QUERY_COL"]] = df[STATE["QUERY_COL"]].astype(str)
        df[STATE["ITEM_COL"]]  = df[STATE["ITEM_COL"]].astype(str)
        df = df.sort_values([STATE["QUERY_COL"], "score", STATE["ITEM_COL"]], ascending=[True, False, True])
        df[STATE["RANK_COL"]] = df.groupby(STATE["QUERY_COL"])["score"].rank(ascending=False, method="first").astype("int32")
        df = df[df[STATE["RANK_COL"]]<=STATE["K_OUT"]][[STATE["QUERY_COL"], STATE["ITEM_COL"], STATE["RANK_COL"]]].reset_index(drop=True)

        # Сохраняем напрямую
        A = art_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        ens_id = Path(DATA["ens_dir"]).name
        tag = now_tag()
        h = short_hash(f"panic_{ens_id}_{STATE['K_OUT']}_{tag}")
        submit_id = f"{tag}_{ens_id}_panic_{STATE['K_OUT']}_{h}"
        out_dir = A["submits"]/submit_id
        out_dir.mkdir(parents=True, exist_ok=True)
        file_path = out_dir/"submission.csv"
        df.to_csv(file_path, index=False)
        save_json(out_dir/"manifest.json", dict(
            submit_id=submit_id, panic=True, dataset_id=STATE["DATASET_ID"], profile=STATE["PROFILE_PATH"],
            ensemble_id=ens_id, format="pairs_csv", K_OUT=STATE["K_OUT"], file=str(file_path), file_fp=fingerprint(file_path),
            created_utc=datetime.utcnow().isoformat()+"Z"
        ))
        if STATE["COMPRESS_ZIP"]:
            zpath = out_dir/"submission.zip"
            with zipfile.ZipFile(zpath, "w", compression=zipfile.ZIP_DEFLATED) as zf:
                zf.write(file_path, arcname=file_path.name)
                zf.write(out_dir/"manifest.json", arcname="manifest.json")
            print("✓ PANIC saved & zipped:", zpath)
        else:
            print("✓ PANIC saved:", file_path)

btn_panic.on_click(on_panic)
W.VBox([btn_panic, out_panic])


In [ ]:
btn_report = W.Button(description="Собрать итоговый отчёт сабмита", button_style="", layout=BTN_LAYOUT)
out_report = W.Output()

def on_report(_):
    with out_report:
        clear_output()
        if not PATHS.get("submit_dir"):
            print("Сначала сохраните сабмит.")
            return
        ens_dir = Path(DATA["ens_dir"])
        submit_dir = Path(PATHS["submit_dir"])
        report = dict(
            submit_id = PATHS["submit_id"],
            dataset_id= STATE["DATASET_ID"],
            profile   = STATE["PROFILE_PATH"],
            ensemble_id = ens_dir.name,
            files = dict(submit=PATHS["submit_file"], zip=PATHS.get("zip_file"), manifest=PATHS["manifest"]),
            blender_config = DATA["blender_config"],
            fixed_stats = FIXED.get("report"),
            created_utc = datetime.utcnow().isoformat()+"Z"
        )
        save_json(submit_dir/"submission_report.json", report)
        print("✓ submission_report.json записан:", submit_dir/"submission_report.json")
        display(pd.DataFrame([report]).T)

btn_report.on_click(on_report)
W.VBox([btn_report, out_report])


### Памятка
- Убедись, что для **каждого** запроса ровно `K_OUT` позиций, без дублей.
- Идентификаторы строковые (или как требует платформа), нет `NaN/Inf`.
- Формат соответствует регламенту (колонки/поля правильно названы).

**Стоп-флаги**
- Не все queries покрыты.
- Дубликаты `(query_id, item_id)` или дубли `item_id` внутри query.
- Нестабильные тай-брейки (плавающий порядок при равных скорингах).

Если что-то горит — жми **PANIC**: быстрый pairs_csv из уже отсортированного `ranked`.
